In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm.notebook import tqdm
from nasbench import api
from search_spaces import load_nasbench_101
from random_search import run_random_search, random_spec
from neural_predictor import classifier, regressor
from input_preprocessing import preprocess_nasbench

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [ ]:
nasbench = load_nasbench_101()

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


### Two-stage vs One-stage prediction 10 split (N=172)

In [ ]:
N = 172

In [ ]:
def get_N_samples(N):
    models = []

    for _ in range(N):
        while True:
            model = random_spec(nasbench)
            if model not in models:
                models.append(nasbench.query(model))
                break
                
    return preprocess_nasbench(models)

In [ ]:
train_clf_data = get_N_samples(N)
clf = classifier([train_clf_data['X'], train_clf_data['norm_A'], train_clf_data['norm_AT']], train_clf_data['labels'])

In [ ]:
def cascade_prediction(train_data, test_data):
    # training phrase
    filtered_train = clf.predict([train_data['X'], train_data['norm_A'], train_data['norm_AT']])
    filtered_test = clf.predict([test_data['X'], test_data['norm_A'], test_data['norm_AT']])
    
    idx = np.where(filtered_train > 0.50)[0]
    X = train_data['X'][idx]
    A = train_data['norm_A'][idx]
    AT = train_data['norm_AT'][idx]
    labels = train_data['val_acc'][idx]
    
    reg = regressor([X, A, AT], labels)
    
    idx = np.where(filtered_test > 0.50)[0]
    X = test_data['X'][idx]
    A = test_data['norm_A'][idx]
    AT = test_data['norm_AT'][idx]
    labels = test_data['val_acc'][idx]
    
    return reg.evaluate([X, A, AT], labels, verbose=0)

In [ ]:
two_stage_mse = []
one_stage_mse = []

for _ in range(10):
    train_reg_data = get_N_samples(N)
    test_reg_data = get_N_samples(N)
    
    two_stage_mse.append(cascade_prediction(train_reg_data, test_reg_data))
    
    reg = regressor([train_reg_data['X'], train_reg_data['norm_A'], train_reg_data['norm_AT']], train_reg_data['val_acc'])
    one_stage_mse.append(reg.evaluate([test_reg_data['X'], test_reg_data['norm_A'], test_reg_data['norm_AT']], test_reg_data['val_acc'], verbose=0))

In [ ]:
np.average(two_stage_mse), np.average(one_stage_mse)

In [ ]:
LOOPS = 60
MAX_SAMPLES = 500
MAX_TIME_BUDGET = 8e5

In [ ]:
train_data = get_N_samples(N)
reg = regressor([train_data['X'], train_data['norm_A'], train_data['norm_AT']], train_data['val_acc'])

### One-stage

In [12]:
np_val_avg, np_test_avg = [], []
np_val_std, np_test_std = [], []

val_acc, test_acc = [], []
for k in tqdm(range(MAX_SAMPLES - N)):
    K = k + 1

    loop_val, loop_test = [], []
    for _ in range(LOOPS):
        test_models = get_N_samples(N+K)
        pred_acc = reg.predict([test_models['X'], test_models['norm_A'], test_models['norm_AT']]).ravel()
        
        topk_idx = tf.math.top_k(pred_acc, k=K).indices.numpy()
        selected_val = test_models['val_acc'][topk_idx]
        selected_test = test_models['test_acc'][topk_idx]
        
        best_val_idx = np.argmax(selected_val)
        
        loop_val.append(selected_val[best_val_idx])
        loop_test.append(selected_test[best_val_idx])

    val_acc.append(np.mean(loop_val))
    test_acc.append(np.mean(loop_test))
    
    np_val_avg.append(np.max(val_acc))
    np_val_std.append(np.std(loop_val))
    
    np_test_avg.append(np.max(test_acc))
    np_test_std.append(np.std(loop_test))

  0%|          | 0/328 [00:00<?, ?it/s]

In [15]:
np.save('outputs/np_val_avg_by_samples.npy', np_val_avg)
np.save('outputs/np_val_std_by_samples.npy', np_val_std)
np.save('outputs/np_test_avg_by_samples.npy', np_test_avg)
np.save('outputs/np_test_std_by_samples.npy', np_test_std)